In [1]:
patient_file = '/Users/akram/Documents/Final Project/data_analysis/criteria_4_1.parquet'

chart_file = '/Users/akram/Documents/Final Project/data_analysis/chartevents.csv'


Read in files to dataframes

In [2]:
pip install dask pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import dask.dataframe as dd

/Users/akram/Library/Python/3.9/lib/python/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [10]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 26.0 MB 5.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
dtypes = {'itemid': 'int64',
    'caregiver_id': 'float64',
       'value': 'object',
       'valuenum': 'float64',
       'warning': 'float64'}

chartevents_df = dd.read_csv(chart_file, dtype=dtypes)

In [7]:
chartevents_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, subject_id to warning
dtypes: object(4), float64(3), int64(4)

In [8]:
chartevents_df.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.0,mmHg,0.0
1,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59,59.0,mmHg,0.0
2,10000032,29079034,39553978,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63,63.0,mmHg,0.0
3,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045,94,94.0,bpm,0.0
4,10000032,29079034,39553978,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.0,mmHg,0.0


In [12]:
patients_df = pd.read_parquet(patient_file)
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6111 entries, 0 to 16240
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     6111 non-null   int64         
 1   hadm_id                        6111 non-null   int64         
 2   stay_id                        6111 non-null   int64         
 3   ventilation_starttime          6111 non-null   datetime64[ns]
 4   ventilation_endtime            6111 non-null   datetime64[ns]
 5   ventilation_itemid             6111 non-null   int64         
 6   ventilation_ordercategoryname  6111 non-null   object        
 7   extubation_starttime           6111 non-null   datetime64[ns]
 8   extubation_endtime             6111 non-null   datetime64[ns]
 9   extubation_itemid              6111 non-null   int64         
 10  extubation_ordercategoryname   6111 non-null   object        
 11  ventilation_duration 

Set itemid values relating to palliative care

In [13]:
itemid_values = {229150, 229152, 229154, 229155}

Filter for itemid only

In [14]:
filtered_chartevents = chartevents_df[chartevents_df['itemid'].isin(itemid_values)]

Merge filtered chartevents with patient data

In [15]:
# Merge the filtered chartevents data with the patient data
merged_data = filtered_chartevents.merge(patients_df, on='subject_id', how='inner')


In [16]:
# Convert 'charttime' column to datetime
merged_data['charttime'] = dd.to_datetime(merged_data['charttime'])


In [17]:
merged_data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 23 entries, subject_id to anchor_age
dtypes: datetime64[ns](5), object(5), float64(4), int64(9)

In [18]:
merged_data.head()

,subject_id,hadm_id_x,stay_id_x,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,...,ventilation_starttime,ventilation_endtime,ventilation_itemid,ventilation_ordercategoryname,extubation_starttime,extubation_endtime,extubation_itemid,extubation_ordercategoryname,ventilation_duration,anchor_age
0,10011365,26712576,37153661,27115.0,2166-02-19 12:37:00,2166-02-19 12:44:00,229150,Patient / family will have support throughout ...,NaN,NaN,...,2166-02-11 16:15:00,2166-02-14 09:46:00,225792,Ventilation,2166-02-14 09:48:00,2166-02-14 09:49:00,227194,Intubation/Extubation,3931.0,73
1,10011365,26712576,37153661,27115.0,2166-02-19 12:37:00,2166-02-19 12:44:00,229152,Assess ability of caregiver to provide care,NaN,NaN,...,2166-02-11 16:15:00,2166-02-14 09:46:00,225792,Ventilation,2166-02-14 09:48:00,2166-02-14 09:49:00,227194,Intubation/Extubation,3931.0,73
2,10011365,26712576,37153661,27115.0,2166-02-19 12:37:00,2166-02-19 12:44:00,229152,Encourage patient / family members to accept i...,NaN,NaN,...,2166-02-11 16:15:00,2166-02-14 09:46:00,225792,Ventilation,2166-02-14 09:48:00,2166-02-14 09:49:00,227194,Intubation/Extubation,3931.0,73
3,10011365,26712576,37153661,27115.0,2166-02-19 12:37:00,2166-02-19 12:44:00,229152,Encourage patient / family to set and verbaliz...,NaN,NaN,...,2166-02-11 16:15:00,2166-02-14 09:46:00,225792,Ventilation,2166-02-14 09:48:00,2166-02-14 09:49:00,227194,Intubation/Extubation,3931.0,73
4,10011365,26712576,37153661,27115.0,2166-02-19 12:37:00,2166-02-19 12:44:00,229152,Ensure spiritual and cultural needs are met,NaN,NaN,...,2166-02-11 16:15:00,2166-02-14 09:46:00,225792,Ventilation,2166-02-14 09:48:00,2166-02-14 09:49:00,227194,Intubation/Extubation,3931.0,73


Filter so charttime of palliative care event is before the end of ventilation

In [19]:
# Filter the merged data based on the charttime and ventilation_endtime conditions
filtered_patients = merged_data[merged_data['charttime'] < merged_data['ventilation_endtime']]


Remove patients under palliative care

In [20]:
# Get the subject_ids to be removed
subject_ids_to_remove = filtered_patients['subject_id'].compute().unique()

# Remove the patients who meet the criteria from the initial patient data
filtered_patient_data = patients_df[~patients_df['subject_id'].isin(subject_ids_to_remove)]


In [21]:
# Save the filtered patient data to a parquet file
filtered_patient_data.to_parquet('filtered_patients.parquet', index=False)


Now we can carry out simple analysis on the filtered patient data to see how many patients have been removed

In [22]:
# Load filtered patient data
filtered_patient_data_df = pd.read_parquet('filtered_patients.parquet')

filtered_patient_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6048 entries, 0 to 6047
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   subject_id                     6048 non-null   int64         
 1   hadm_id                        6048 non-null   int64         
 2   stay_id                        6048 non-null   int64         
 3   ventilation_starttime          6048 non-null   datetime64[ns]
 4   ventilation_endtime            6048 non-null   datetime64[ns]
 5   ventilation_itemid             6048 non-null   int64         
 6   ventilation_ordercategoryname  6048 non-null   object        
 7   extubation_starttime           6048 non-null   datetime64[ns]
 8   extubation_endtime             6048 non-null   datetime64[ns]
 9   extubation_itemid              6048 non-null   int64         
 10  extubation_ordercategoryname   6048 non-null   object        
 11  ventilation_durat

In [24]:
# Calculate the number of patients who were removed
num_removed_patients = len(subject_ids_to_remove)
print(f'Number of patients under palliative care removed: {num_removed_patients}')

Number of patients under palliative care removed: 63
